## Testing the trained data both using video feed and live demo video

### Step-1: Import important libraries

In [54]:
import cv2
import numpy as np
from keras.models import model_from_json
from google.colab.patches import cv2_imshow

### step-2: Give a lable for catagories

In [2]:
emotion_dict = {0: "Facial Tension", 1: "No Facial Tension"}

### Step-3: load json file and create model

In [3]:
json_file = open('/content/best_model/emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

### Step-4: load weights (.h5 file) into new model

In [4]:
emotion_model.load_weights("/content/best_model/emotion_model.h5")
print("Loaded model from disk")

Loaded model from disk


### Step-5: Start the webcam feed the live video recording

In [22]:
#for live webcam feed: test-1
cap = cv2.VideoCapture(0)

### Step-6: Use custom feed video

In [39]:
#for video path feed: test-2
cap = cv2.VideoCapture("/content/sample_video/sample1.mp4")

### Step-7: Start testing by processing the given video using Haar-algorithm for face detection

In [ ]:
while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1280, 720))
    if not ret:
        break
    face_detector = cv2.CascadeClassifier('/content/haardcascade/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # take each face available on the camera and Preprocess it using a rectangle with gree color
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # predict the emotions
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
   
    #cv2.imshow('Emotion Detection', frame)
    cv2_imshow(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):       # Hit letter q to exit from the image capture
        break

cap.release()
cv2.destroyAllWindows()